# Sistemas inteligentes para respostas a perguntas médicas

Gyovana M. Moriyama (216190)

Rafael A. Matumoto (273085)

In [1]:
!pip install -qU langchain_openai langchain-community faiss-cpu sentence-transformers openai datasets pydantic langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

## Experiments with prompts

- Embedding model: avsolatorio/GIST-small-Embedding-v0
- Text splitter: chunk_size=300, chunk_overlap=30
- Retriever: FAISS

Test variations of ReAct prompts and check if improvements in accuracy can be achieved with these changes.

In [2]:
import os
import datasets
import datetime
import re
import pandas as pd

from google.colab import userdata, drive

from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List, Optional, Literal

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_core.load import dumps, loads

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.environ['OPENAI_API_KEY'] = userdata.get('api_key')

In [5]:
folder_path = '/content/drive/MyDrive/IA024A_Processamento_de_Linguagem_Natural/Projeto Final/projeto/entrega4/'

### MedQA-USMLE-4-options dataset

In [6]:
# run experiments on train split
data = datasets.load_dataset('GBaker/MedQA-USMLE-4-options', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [7]:
data = data.add_column('original_quest_id', range(len(data)))

In [8]:
# sample questions for experiments
sampled_data = data.shuffle(seed=42).select(range(50))

### ReAct agent

In [9]:
embed_model = 'avsolatorio/GIST-small-Embedding-v0'

embedding_model = HuggingFaceBgeEmbeddings(model_name=embed_model)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# load saved faiss indexes
retriever = f'{folder_path}retrievers/faiss__300_30__20241110_163802'

faiss = FAISS.load_local(
    folder_path=retriever,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# create a search tool from the faiss retriever
tool_search_faiss = create_retriever_tool(
    retriever=faiss.as_retriever(search_kwargs={'k': 3}),
    name='search_docs',
    description='Searches the query in documents and returns the top-3 most relevant ones.',
)

In [11]:
model = 'gpt-4o-mini'
model_temp = 0.5

In [ ]:
# react prompt  ORIGINAL
# react_prompt_template = '''
# Solve the question answering task alternating between Question, Thought, Action, Input and Observation steps.
# You only have access to the following tools: {tools}

# Use the following format:
# Question: the input question you must answer
# Thought: you should always think about what to do. Break the problem down into subproblems and smaller steps and decide which action to take.
# Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
# Action Input: the input to the action
# Observation: the output of the action.
# ... (this Thought/Action/Action Input/Observation can repeat any number of times)
# Final Answer: A/B/C/D

# Begin!
# Question: {question}
# Options: {options}
# Thought: {agent_scratchpad}
# '''.strip()

In [ ]:
react_prompt_template = '''
Perform the question-answering task using a structured approach. Alternate between Question, Thought, Action, Input, and Observation steps to reach a well-reasoned answer.

You have access to the following tools: {tools}

Use the following format:

Question: The medical question you must answer
Thought: Reflect on the question, analyze key aspects, and identify subproblems. Decide on the best approach, whether gathering information, verifying clinical guidelines, or recalling medical knowledge.
Action: Specify the action to take, chosen from [{tool_names}]. If no action is needed, proceed to provide the Final Answer.
Action Input: The specific input or query for the chosen action.
Observation: The output from the action or tool. ... (Repeat Thought/Action/Action Input/Observation as needed)
Final Answer: A/B/C/D
Begin!

Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt_template2 = '''
Solve the medical question-answering task by alternating between Question, Thought, Action, Action Input, and Observation steps.
You only have access to the following tools: {tools}.

Use the following format:

Question: The medical question you need to answer.
Thought: Think critically about the problem, breaking it down into relevant medical subtopics and smaller steps. Take into consideration the given options. Determine the appropriate next action.
Action: The action to take. This should be one of [{tool_names}]. If you determine no additional action is necessary, proceed to provide the Final Answer.
Action Input: The input required for the chosen action.
Observation: The result or output from the action taken.
... (this Thought/Action/Action Input/Observation sequence may repeat as necessary)
Final Answer: Choose the best answer from options A/B/C/D.

Begin your process below!

Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [ ]:
react_prompt_template4 = '''
Solve the medical question-answering task by alternating between Question, Thought, Action, Action Input, and Observation steps.
You only have access to the following tools: {tools}.

Use the following format:

Question: The medical question you need to answer.
Thought: think step by step to reach a well-reasoned answer.
Action: The action to take. This should be one of [{tool_names}]. If no action is necessary, proceed to provide the Final Answer.
Action Input: The input required for the chosen action. Limit your query into 3 terms.
Observation: The result or output from the action taken.
... (this Thought/Action/Action Input/Observation sequence may repeat as necessary)
Final Answer: Choose the best answer from options A/B/C/D.

Begin your process below!

Question: {question}
Options: {options}
Thought: {agent_scratchpad}
'''.strip()

In [12]:
# act only prompt
react_prompt_template_act = '''
Solve the question answering task alternating between Question, Action, Input and Observation steps.
You have access to the following tools:{tools}

Use the following format:

Question: the input question you must answer
Action: the action to take, must be one of [{tool_names}]. If no action is needed, return your Final answer instead.
Action Input: the input to the action
Observation: what you infer from the result of the action
... (this Action/Action Input/Observation can repeat any number of times)
Final Answer: A/B/C/D

Begin!
Question: {question}
Options: {options}
{agent_scratchpad}
'''.strip()

In [13]:
react_prompt = ChatPromptTemplate.from_template(react_prompt_template_act)

In [14]:
llm = ChatOpenAI(
        model=model,
        temperature=model_temp,
    )

In [15]:
# create agent with tools
agent = create_react_agent(
    llm=llm,
    tools=[tool_search_faiss],
    prompt=react_prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[tool_search_faiss],
    verbose=False,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
    max_iterations=5
)

In [16]:
agent_executor.invoke({'question': sampled_data[0]['question'], 'options': sampled_data[0]['options']})

{'question': 'A 35-year-old woman comes to your office with a variety of complaints. As part of her evaluation, she undergoes laboratory testing which reveals the presence of anti-centromere antibodies. All of the following symptoms and signs would be expected to be present EXCEPT:',
 'options': {'A': 'Pallor, cyanosis, and erythema of the hands',
  'B': 'Blanching vascular abnormalities',
  'C': 'Hypercoagulable state',
  'D': 'Heartburn and regurgitation'},
 'output': 'C',
 'intermediate_steps': [(AgentAction(tool='search_docs', tool_input='anti-centromere antibodies symptoms and signs', log='Action: search_docs  \nAction Input: "anti-centromere antibodies symptoms and signs"  '),
   'These patients may have anticentromere antibodies.\n\nhas been termed the CREST syndrome. Centromere antibodies have been reported in a very high percentage of patients with the CREST syndrome but in only a small minority of patients with scleroderma. Skin biopsy reveals thickening of the dermis and hom

In [17]:
tmp = retriever.split('/')[-1].split('__')[1]
filename = f'{folder_path}results/answers_faiss_promptB_act_{tmp}_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'

# run agent
with open(filename, 'w') as f:

    f.write(''.center(10, '-'))
    f.write('\n')
    f.write(f'Model: {model}\n')
    f.write(f'Temperature: {model_temp}\n')
    f.write(f'Embedding model: {embed_model}\n')
    f.write(f'Prompt: {react_prompt}\n')
    f.write(''.center(10, '-'))
    f.write('\n[{')

    for n, question in enumerate(tqdm(sampled_data)):

        res = agent_executor.invoke({'question': question['question'], 'options': question['options']})
        f.write(f'"{n}": ')

        # serialize results as a string
        f.write(dumps(res))
        f.write(',')

    f.write('"placeholder": "placeholder"') # add a placeholder k:v pair for trailling comma above
    f.write('}]')

100%|██████████| 50/50 [13:31<00:00, 16.24s/it]


#### Results with FAISS in 50 sample questions

In [18]:
os.listdir(f'{folder_path}results/')


['answers_faiss_600_0_20241110_130142.txt',
 'direct_sample_20241110_204427.txt',
 'cot_sample_20241110_214655.txt',
 'direct_sample_20241110_215410.txt',
 'direct_sample_20241110_215443.txt',
 'direct_sample_20241110_215700.txt',
 'direct_sample_20241110_215734.txt',
 'direct_sample_20241110_220451.txt',
 'cot_sample_20241110_220531.txt',
 'react_sample_20241110_223744.txt',
 'answers_faiss_prompt2_300_30_20241112_001640.txt',
 'answers_faiss_promptB_300_30_20241112_002531.txt',
 'answers_faiss_prompt2_300_30_20241112_002650.txt',
 'answers_faiss_prompt2_300_30_20241112_003236.txt',
 'answers_faiss_300_30_20241110_170548.txt',
 'answers_faiss_prompt3_300_30_20241112_005805.txt',
 'answers_faiss_promptB_300_30_20241112_005855.txt',
 'answers_faiss_prompt4_300_30_20241112_011058.txt',
 'answers_faiss_promptB2_300_30_20241112_012242.txt',
 'answers_faiss_prompt6_300_30_20241112_013420.txt',
 'answers_faiss_prompt5_300_30_20241112_012059.txt',
 'answers_faiss_promptB3_300_30_20241112_0137

In [19]:
result_logs = [
    f'{folder_path}results/answers_faiss_300_30_20241110_170548.txt',
    f'{folder_path}results/answers_faiss_promptB_300_30_20241112_002531.txt',
    f'{folder_path}results/answers_faiss_promptB2_300_30_20241112_012242.txt',
    f'{folder_path}results/answers_faiss_promptB4_300_30_20241112_015232.txt',
    f'{folder_path}results/answers_faiss_promptB_act_300_30_20241113_012301.txt',
]

In [20]:
acc_df = pd.DataFrame(columns=['Chunk Size', 'Overlap Size', 'Prompt', 'Accuracy'])

for logfile in result_logs:

    chunk_size = logfile.split('/')[-1].split('_')[-4]
    chunk_overlap = logfile.split('/')[-1].split('_')[-3]
    prompt_version = logfile.split('/')[-1].split('_')[-5]
    prompt_version = 'prompt' if prompt_version == 'faiss' else prompt_version

    print(chunk_size, chunk_overlap, prompt_version)

    with open(logfile) as f:

        # load log file with LLM results
        tmp = f.readlines()

    try:

        # recover results as a dictionary
        recovered_results = loads(tmp[-1])[0]

        limit_error = list()
        not_parsed = list()
        answers = list()

        # matches literals A, B, C, or D at the beginning of a sentence (followed by the text of the corresponding option)
        # e.g. A (answer text) => A
        # other cases will be ignored
        ans_ptn = re.compile(r'(^[A|B|C|D])\s\(')

        for n, (k, v) in enumerate(recovered_results.items()):

            # skip placeholder key
            if k == 'placeholder':
                continue

            # max_iteration set to 5
            if v['output'] == 'Agent stopped due to iteration limit or time limit.':
                answers.append(None)
                limit_error.append(k)

            elif len(v['output']) > 1:
                tmp_output = ans_ptn.findall(v['output'])
                if len(tmp_output) == 1:
                    answers.append(tmp_output[0])
                elif len(tmp_output) > 1:
                    answers.append(None)
                    not_parsed.append(k)
                    print(tmp_output)
                else:
                    answers.append(None)
                    not_parsed.append(k)
                    print(k, v['output'])

            else:
                if v['output'] in ['A', 'B', 'C', 'D']:
                    answers.append(v['output'])
                else:
                    answers.append(None)

        # unanswered questions or answers not properly parsed
        print('Unanswered questions or answers not properly parsed: ', len(limit_error))

        # parsing errors/other errors
        print('Parsing errors/other errors: ', len(not_parsed))

        # accuracy
        accuracy = sum([pred == gt for gt, pred in zip(sampled_data['answer_idx'], answers)]) / len(sampled_data)
        print('Accuracy: ', accuracy)
        print('\n')

    except:

        accuracy = None


    acc_df.loc[len(acc_df)] = [chunk_size, chunk_overlap, prompt_version, accuracy]

300 30 prompt


<ipython-input-20-deadc276e146>:20: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  recovered_results = loads(tmp[-1])[0]


Unanswered questions or answers not properly parsed:  1
Parsing errors/other errors:  0
Accuracy:  0.78


300 30 promptB
35 ** C
48 **B** - Increased glutamate concentration.
Unanswered questions or answers not properly parsed:  26
Parsing errors/other errors:  2
Accuracy:  0.34


300 30 promptB2
26 **A**
40 **C** - Regions of hyperdensity in the left pons.
Unanswered questions or answers not properly parsed:  22
Parsing errors/other errors:  2
Accuracy:  0.42


300 30 promptB4
1 B: 'GAA'
3 D: Cor pulmonale
5 C: IV acyclovir
8 C: Herpes simplex virus
10 D: Enterocele
11 B: Lysyl oxidase
13 D: 'Replication of the attenuated vaccine strain'
14 B: 'Lewy bodies'
15 B: Small cell lung carcinoma
17 D: Terminal bronchioles
19 B: Rotator cuff tear
21 B. Calcitriol
22 C: Hemangioblastoma
23 D: 'HLA-DR4'
24 A. Autosomal dominant polycystic kidney disease (ADPKD)
29 A: Achlorhydria
30 A: Mesenteric angiography.
33 A: 'Antibiotics + CT-guided drainage'
35 C: Intestinal obstruction
36 D: Resistance

In [21]:
acc_df

,Chunk Size,Overlap Size,Prompt,Accuracy
0,300,30,prompt,0.78
1,300,30,promptB,0.34
2,300,30,promptB2,0.42
3,300,30,promptB4,0.28
4,300,30,act,0.10
